#  <center>Practical Bayesian Optimization of Machine Learning Algorithm </center>
<b> Vincent LE MEUR, Thomas Levy,Timothée Watrigant </b>
### <center><b></b></center>

# Sources

L'article étudié est le suivant : https://papers.nips.cc/paper/4522-practical-bayesian-optimization-of-machine-learning-algorithms.pdf

Pour installer un kernel python 2 sur jupyter
http://ipython.readthedocs.io/en/stable/install/kernel_install.html

Le code de l'implémentation est : 
https://github.com/JasperSnoek/spearmint

# Introduction

Le choix des hyper-paramètres (ex : taux d'apprentissage, nombre de couches d'un réseau de neuronnes...) d'un modèle de Machine Learning peut être fastidieux et relève plus
souvent de l’expérience empirique que d'une méthode exacte. 

Cette optimisation peut être vue comme celle d'une fonction inconnue $f(x)$ qui évalue l'efficacité d'un algorithme avec des hyperparamètres $x$ fixés.

L'article propose un algorithme basé
sur l'optimisation bayésienne avec un prior Processus Gaussien (GP) pour automatiser le choix des
hyper-paramètres. Cet algorithme a été appliqué et validé sur 3 problèmes de Machine Learning.

Nous allons décrire le principe de l'optimization Bayésienne, puis l'algorithme utilisé par les auteurs de l'étude. Enfin, nous essaierons de l'appliquer à un nouveau problème de
machine learning.

# 1) L'optimisation Bayésienne

L'idée est de considérer que la fonction inconnue $f(x)$ est issue d'un Prior et dont la distribution a posteriori reste cohérente avec ce Prior au fur et à mesure des observations. L'objectif est de trouver le minimum de cette fonction $f : \mathcal{X} \rightarrow \mathcal{R}$ où $\mathcal{X}$ désigne l'ensemble dans lequel évolue les hyperparamètres.

Dans notre cas précis, une observation correspond à faire tourner un algorithme de Machine Learning avec un ensemble d'hyperparamètres fixés et d'en évaluer l'efficacité.

L'optimisation bayésienne permet de choisir de manière "optimale" le prochain set d'hyperparamètres à tester. En effet, cette approche intégre l'incertitude sur la fonction $f$ de façon à la réduire lors de la prochaine évaluation. Ainsi comme dans beaucoup d'approches bayésiennes, l'ensemble de l'information des données est utilisée.

Pour pratiquer de l'optimisation bayésienne, il y a deux choix principaux à réaliser.
La premier choix est celui du <b>Prior</b> concernant la fonction $f$.
Le second choix est celui de la <b>fonction d'acquisition</b>. Cette dernière va servir à évaluer le prochain point à tester.

Les auteurs de l'article ont choisi comme prior un <b>Processus Gaussien</b> qui offre une excellente flexibilité sur la fonction $f$. Ainsi par définition, nos observations obtenues seront de la forme : $ \{ x_n,y_n\}$ avec $y_n \sim \mathcal{N}(f(x_n),\nu)$ avec $\nu$ la variance du bruit de mesure des observations.
Le support d'un tel processus peut se résumer avec deux fonctions : 
$m : \mathcal{X} \rightarrow \mathcal{R}$ la fonction moyenne, et $K : \mathcal{X} \times \mathcal{X} \rightarrow \mathcal{R} $ la fonction de covariance

La fonction d'acquisition quand à elle désigne une fonction $a : \mathcal{X} \rightarrow \mathcal{R}^+ $ qui nous permet de choisir le prochain point d'évaluation : $x_{next} = argmax_{\mathcal{X}}a(x)$. Ces fonctions dépendent des paramètres du Processus Gaussien ainsi que des précédentes observations. Plusieurs fonctions sont alors possibles comme "Probability of Improvment" qui maximise la probabilité d'avoir une "meilleure observation" par rapport à toutes les observations précédentes ou encore le "GP Upper Confidence Bound". Le choix des auteurs de l'article s'est porté sur l' <b>Expected Improvment $a_{EI}$</b>

Soit $\theta$ les hyperparamètres de notre GP, $\mu$ la fonction moyenne de prédiction et $\sigma$ la fonction de variance sous le prior GP (à ne pas confondre avec les fonctions $m$ et $K$ précédentes),$x_{best}=argmin_{x_1,..,x_n}f(x)$ la meilleure observation actuelle, $\gamma(x) = \dfrac{f(x_{best})-\mu(x; \{x_n,y_n\},\theta)}{\sigma(x;\{x_n,y_n\},\theta)}$ et $\Phi$ la fonction de répartition de la loi Normale alors l'Expected Improvment est définie par : 


$a_{EI}(x; \{x_n,y_n\},\theta) = \sigma(x; \{x_n,y_n\},\theta)(\gamma(x)\Phi(x) + \mathcal{N}(\gamma(x);1))$


Ce choix est justifié par de bonnes performances en minimisation et l'absence d'hyperparamètres supplémentaires.

# 2) Points clés de l'étude

Le travail réalisé dans ce contexte s'attaque à trois difficultés précises. La première est celle du choix de la fonction de covariance $K$ du GP qui peut être déterminant d'un problème à l'autre. La seconde est la volonté de prendre en compte le temps d'évaluation de la fonction $f$ qui peut changer radicalement d'un problème à l'autre. En effet, on rappelle que l'évaluation de cette fonction implique la mise en place d'un algorithme de Machine Learning complet. Enfin, malgré la nature séquentielle de cette optimisation bayésienne, les auteurs ont cherché à tirer profit de la parallélisation des calculs propres aux environnements distribués.

##  a) Le choix de la fonction de covariance

Le choix de cette fonction covariance peut induire d'importantes hypothèses sur la fonction $f$

Un choix courant est celui du "squared exponential kernel" (ou ARD). Néanmoins, les fonctions obtenues pour ce choix ont une régularité importante. Bien que cela soit positif d'un point de vue de l'optimisation, cela reste très peu réaliste pour l'optimisation complexe de notre fonction $f$.

C'est pourquoi les auteurs ont plutôt choisi comme fonction de covariance un choix plus "exotique", l'ARD Matérn 5/2 kernel :

$ K_{M52}(x,x') = \theta_0(1+ \sqrt{5r^2(x,x')} + \frac{5}{3}r^2(x,x'))e^{-\sqrt{5r^2(x,x')}} $
avec $ r^2(x,x')=\sum_{d=1}^D(x_d - x_d')^2/\theta^2_d$

On constate alors la présence de D+3 hyperparamètres issue de notre processus Gaussien (D est la dimension des vecteurs $x \in \mathcal{X}$) :
- D paramètres de longeurs d'échelles $\theta_{1:D}$
- $\theta_0$ l'amplitude de la covariance 
- $\nu$ le bruit de mesure et $m$ la moyenne

On concatène alors tous ces paramètres en un unique vecteur $\theta$

Il y a donc D+3 hyperparamètres à sélectionner. Pour avoir un traitement bayésien complet, l'idée des auteurs a été de modifier la fonction d'acquisition $a$ précédente en intégrant selon ce vecteur $\theta$. On obtient alors la fonction d'acquisition intégrée suivante : 

$â(x;\{x_n,y_n\}) = \int a(x:\{x_n,y_n\},\theta)p(\theta | \{x_n,y_n\}_{n=1}^N)d\theta $ 

avec $p(\theta | \{x_n,y_n\}_{n=1}^N)$ la distribution marginale issue des données et du processus Gaussien.

Il s'agit d'une généralisation permettant de prendre en compte l'incertitude sur le choix des hyperparamètres du GP.

## b) La prise en compte du temps d'évaluation

Bien que l'optimisation bayésienne précédente nous permette d'effectuer un choix optimal pour la prochaîne évaluation de f, cela peut se solder en pratique par un temps d'exécution très long (cela dépend fortement de la nature de l'espace des hyperparamètres $\mathcal{X}$). Cela rendrait donc l'étude non applicable en pratique avec des configurations machines classiques.

La fonction $f$ nous est inconnue tout comme la fonction $c : \mathcal{X} \rightarrow \mathcal{R}^+$ qui évalue le temps d'évaluation de la fonction $f$ au point $x$. 

Le point clé ici est d'utiliser toute la machinerie de notre optimisation bayésienne pour évaluer $ln$ $c(x)$ en plus de $f(x)$ en supposant que ces deux fonctions sont indépendantes l'une de l'autre.

Ainsi il est possible de tracer l'"expected improvment per second" en divisant par cette évaluation.

L'algorithme nous permet non seulement de choisir des points qui donneront une bonne optimisation de f mais également des points pour lesquels cette évaluation n'est pas trop couteuse en temps d'exécution.

## c) La parallélisation de l'optimisation Bayésienne par des acquisitions de Monte Carlo

Bon cette partie je l'ai toujours pas comprise faut que je creuse 

# 3) Implémentation 

In [18]:
# Si il y a des problèmes pour importer numpy et scipy sur ce kernel les deux commandes ci-dessous devraient régler le problème
#import sys
#!conda install --yes --prefix {sys.prefix} numpy
#!{sys.executable} -m pip install scipy
#!{sys.executable} -m pip install weave

### Fonctions utilisées pour le GP (contenues dans gp.py) : 

In [2]:
import numpy as np
import scipy.linalg as spla
import scipy.optimize as spo
import scipy.io as sio
import weave
try:
    import matplotlib.pyplot as plt
except:
    pass

SQRT_3 = np.sqrt(3.0)
SQRT_5 = np.sqrt(5.0)

def dist2(ls, x1, x2=None):
     # Assumes NxD and MxD matrices.
     # Compute the squared distance matrix, given length scales.
     
    if x2 is None:
        
         # Find distance with self for x1.
 
         # Rescale.
        xx1 = x1 / ls        
        xx2 = xx1
 
    else:
        
        
         # Rescale.
        xx1 = x1 / ls
        xx2 = x2 / ls
     
    r2 = np.maximum(-(np.dot(xx1, 2*xx2.T) 
                        - np.sum(xx1*xx1, axis=1)[:,np.newaxis]
                        - np.sum(xx2*xx2, axis=1)[:,np.newaxis].T), 0.0)
 
    return r2


def grad_dist2(ls, x1, x2=None):
    if x2 is None:
        x2 = x1
    # Rescale.
    x1 = x1 / ls
    x2 = x2 / ls
     
    N = x1.shape[0]
    M = x2.shape[0]
    D = x1.shape[1]
    gX = np.zeros((x1.shape[0],x2.shape[0],x1.shape[1]))
 
    code = \
    """
    for (int i=0; i<N; i++)
        for (int j=0; j<M; j++)
            for (int d=0; d<D; d++)
               gX(i,j,d) = (2/ls(d))*(x1(i,d) - x2(j,d));
     """
    try:
        scipy.weave.inline(code, ['x1','x2','gX','ls','M','N','D'], \
                        type_converters=scipy.weave.converters.blitz, \
                        compiler='gcc')
    except:
        # The C code weave above is 10x faster than this:
        for i in xrange(0,x1.shape[0]):
            gX[i,:,:] = 2*(x1[i,:] - x2[:,:])*(1/ls)
 
    return gX

def SE(ls, x1, x2=None, grad=False):
    ls = np.ones(ls.shape)
    cov = np.exp(-0.5 * dist2(ls, x1, x2))
    if grad:
        return (cov, grad_ARDSE(ls, x1, x2))
    else:
        return cov    
    
def ARDSE(ls, x1, x2=None, grad=False):
    cov = np.exp(-0.5 * dist2(ls, x1, x2))
    if grad:
        return (cov, grad_ARDSE(ls, x1, x2))
    else:
        return cov

def grad_ARDSE(ls, x1, x2=None):
    r2 = dist2(ls, x1, x2)
    r  = np.sqrt(r2)
    return -0.5*np.exp(-0.5*r2)[:,:,np.newaxis] * grad_dist2(ls, x1, x2)

def Matern32(ls, x1, x2=None, grad=False):
    r   = np.sqrt(dist2(ls, x1, x2))
    cov = (1 + SQRT_3*r) * np.exp(-SQRT_3*r)
    if grad:
        return (cov, grad_Matern32(ls, x1, x2))
    else:
        return cov


def grad_Matern32(ls, x1, x2=None):
    r       = np.sqrt(dist2(ls, x1, x2))
    grad_r2 = -1.5*np.exp(-SQRT_3*r)
    return grad_r2[:,:,np.newaxis] * grad_dist2(ls, x1, x2)
 
def Matern52(ls, x1, x2=None, grad=False):
    r2  = np.abs(dist2(ls, x1, x2))
    r   = np.sqrt(r2)
    cov = (1.0 + SQRT_5*r + (5.0/3.0)*r2) * np.exp(-SQRT_5*r)
    if grad:
        return (cov, grad_Matern52(ls, x1, x2))
    else:
        return cov


def Matern52(ls, x1, x2=None, grad=False):
    r2  = np.abs(dist2(ls, x1, x2))
    r   = np.sqrt(r2)
    cov = (1.0 + SQRT_5*r + (5.0/3.0)*r2) * np.exp(-SQRT_5*r)
    if grad:
        return (cov, grad_Matern52(ls, x1, x2))
    else:
        return cov

def grad_Matern52(ls, x1, x2=None):
    r       = np.sqrt(dist2(ls, x1, x2))
    grad_r2 = -(5.0/6.0)*np.exp(-SQRT_5*r)*(1 + SQRT_5*r)
    return grad_r2[:,:,np.newaxis] * grad_dist2(ls, x1, x2)


class GP:
    def __init__(self, covar="Matern52", mcmc_iters=10, noiseless=False):
        self.cov_func        = globals()[covar]
        self.mcmc_iters      = int(mcmc_iters)
        self.D               = -1
        self.hyper_iters     = 1
        self.noiseless       = bool(int(noiseless))
        self.hyper_samples = []
        
        self.noise_scale = 0.1  # horseshoe prior 
        self.amp2_scale  = 1    # zero-mean log normal prior
        self.max_ls      = 2    # top-hat prior on length scales 

    def real_init(self, dims, values):
        # Input dimensionality. 
        self.D = dims

        # Initial length scales.               
        self.ls = np.ones(self.D)

        # Initial amplitude.        
        self.amp2 = np.std(values)

        # Initial observation noise.                                          
        self.noise = 1e-3

        # Initial mean.
        self.mean = np.mean(values)

    def cov(self, x1, x2=None):
        if x2 is None:
            return self.amp2 * (self.cov_func(self.ls, x1, None)
                                + 1e-6*np.eye(x1.shape[0]))
        else:
            return self.amp2 * self.cov_func(self.ls, x1, x2)

    def logprob(self, comp, vals):
            mean  = self.mean
            amp2  = self.amp2
            noise = self.noise
            
            cov   = amp2 * (self.cov_func(self.ls, comp, None) + 1e-6*np.eye(comp.shape[0])) + noise*np.eye(comp.shape[0])
            chol  = spla.cholesky(cov, lower=True)
            solve = spla.cho_solve((chol, True), vals - mean)
            lp    = -np.sum(np.log(np.diag(chol)))-0.5*np.dot(vals-mean, solve)
            return lp

    def optimize_hypers(self, comp, vals):
        self.mean = np.mean(vals)
        diffs     = vals - self.mean

        state = { }

        def jitter_chol(covmat):
            passed = False
            jitter = 1e-8
            val = 0
            while not passed:
                if (jitter > 100000):
                    val = spla.cholesky(np.eye(covmat.shape[0]))
                    break
                try:
                    val = spla.cholesky(covmat +
                        jitter*np.eye(covmat.shape[0]), lower=True)
                    passed = True
                except ValueError:
                    jitter = jitter*1.1
                    print "Covariance matrix not PSD, adding jitter:", jitter
                    passed = False
            return val
        
        def memoize(amp2, noise, ls):
            if ( 'corr' not in state
                 or state['amp2'] != amp2
                 or state['noise'] != noise
                 or np.any(state['ls'] != ls)):

                # Get the correlation matrix
                (corr, grad_corr) = self.cov_func(ls, comp, None, grad=True)
        
                # Scale and add noise & jitter.
                covmat = (amp2 * (corr + 1e-6*np.eye(comp.shape[0])) 
                          + noise * np.eye(comp.shape[0]))

                # Memoize
                state['corr']      = corr
                state['grad_corr'] = grad_corr
                state['chol']      = jitter_chol(covmat)
                state['amp2']      = amp2
                state['noise']     = noise
                state['ls']        = ls
                
            return (state['chol'], state['corr'], state['grad_corr'])

        def nlogprob(hypers):
            amp2  = np.exp(hypers[0])
            noise = np.exp(hypers[1])
            ls    = np.exp(hypers[2:])

            chol  = memoize(amp2, noise, ls)[0]
            solve = spla.cho_solve((chol, True), diffs)
            lp    = -np.sum(np.log(np.diag(chol)))-0.5*np.dot(diffs, solve)
            return -lp
        def grad_nlogprob(hypers):
            amp2  = np.exp(hypers[0])
            noise = np.exp(hypers[1])
            ls    = np.exp(hypers[2:])

            chol, corr, grad_corr = memoize(amp2, noise, ls)
            solve   = spla.cho_solve((chol, True), diffs)
            inv_cov = spla.cho_solve((chol, True), np.eye(chol.shape[0]))

            jacobian = np.outer(solve, solve) - inv_cov

            grad = np.zeros(self.D + 2)

            # Log amplitude gradient.
            grad[0] = 0.5 * np.trace(np.dot( jacobian, corr + 1e-6*np.eye(chol.shape[0]))) * amp2

            # Log noise gradient.
            grad[1] = 0.5 * np.trace(np.dot( jacobian, np.eye(chol.shape[0]))) * noise

            # Log length scale gradients.
            for dd in xrange(self.D):
                grad[dd+2] = 1 * np.trace(np.dot( jacobian, -amp2*grad_corr[:,:,dd]*comp[:,dd][:,np.newaxis]/(np.exp(ls[dd]))))*np.exp(ls[dd])

            # Roll in the prior variance.
            #grad -= 2*hypers/self.hyper_prior

            return -grad
        
        # Initial length scales.
        self.ls = np.ones(self.D)
        # Initial amplitude.
        self.amp2 = np.std(vals)
        # Initial observation noise.
        self.noise = 1e-3
        
        hypers     = np.zeros(self.ls.shape[0]+2)
        hypers[0]  = np.log(self.amp2)
        hypers[1]  = np.log(self.noise)
        hypers[2:] = np.log(self.ls)
        
        # Use a bounded bfgs just to prevent the length-scales and noise from 
        # getting into regions that are numerically unstable
        b = [(-10,10),(-10,10)]
        for i in xrange(comp.shape[1]):
            b.append((-10,5))
  
        hypers = spo.fmin_l_bfgs_b(nlogprob, hypers, grad_nlogprob, args=(), bounds=b, disp=0)
                
        #hypers = spo.fmin_bfgs(nlogprob, hypers, grad_nlogprob, maxiter=100)
        hypers = hypers[0]
        #hypers = spo.fmin_bfgs(nlogprob, hypers, grad_nlogprob, maxiter=100)

        self.amp2  = np.exp(hypers[0])
        self.noise = np.exp(hypers[1])
        self.ls    = np.exp(hypers[2:])
